In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.model_selection import train_test_split
from utils import load_galaxy_data

import app



input_data, labels = load_galaxy_data()
print(input_data.shape)
print(labels.shape)

#  splitting the dataset
x_train,x_test,y_train,y_test = train_test_split(input_data, labels, test_size = 0.20, shuffle = True, random_state = 222, stratify=labels)

data_generator = ImageDataGenerator(rescale=1./255)
training_iterator = data_generator.flow(x_train, y_train,batch_size=5)

BATCH_SIZE = 5
validation_iterator = data_generator.flow(x_test, y_test, batch_size=BATCH_SIZE)

# build your model, starting with the input shape and output layer:
model = tf.keras.Sequential()

# set the input layer:
model.add(tf.keras.Input(
  shape = input_data.shape[1:]
))

# Configuring Convolutional Layer 
model.add(tf.keras.layers.Conv2D(
  8, 
  3, 
  strides = 2,
  activation = "relu"
)) 

model.add(tf.keras.layers.MaxPooling2D(
  pool_size = (2, 2),
  strides = (2, 2)
))

model.add(tf.keras.layers.Conv2D(
  8, 
  3, 
  strides = 2,
  activation = "relu"
)) 

model.add(tf.keras.layers.MaxPooling2D(
  pool_size = (2, 2),
  strides = (2, 2)
))
# Flatten Layer
model.add(tf.keras.layers.Flatten())
# set the hidden layer
Dense = tf.keras.layers.Dense
model.add(Dense(
  16, 
  activation = "relu" 
))
# set the output layer:
model.add(Dense(
  4, 
  activation = "softmax"
))

# At this point, your model should have 7,164 parameters. 
#model.summary()

# compile your model:
model.compile(
  loss = tf.keras.losses.CategoricalCrossentropy(), 
  optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001), 
  metrics = [tf.keras.metrics.CategoricalAccuracy(), tf.keras.metrics.AUC()]
)

# Train and evaluate the classification model
# train your model:
model.fit(
  training_iterator, 
  steps_per_epoch = len(training_iterator.x) / BATCH_SIZE,
  epochs = 8,
  validation_data = validation_iterator,
  validation_steps = len(validation_iterator.x) / BATCH_SIZE
)

# visualize how your convolutional neural network processes images
visualize_activations(model,validation_iterator)